# IBM Applied Data Science Capstone Project

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Assignment 1</a>

2. <a href="#item2">Assignment 2</a>   
    
</font>
</div>

<a id='item1'></a>
# Assignment 1
Notebook setup and post to Github

In [1]:
# Initial setup of libraries
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<a id='item2'></a>
# Assignment 2
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto  
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.  
Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.  


## 1. Wrangle data: Scrape wiki article
Ref: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  

In [3]:
from bs4 import BeautifulSoup
import requests

Scrape the wiki article:

In [4]:
#URL to scrape
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

# Verify the scrape
print("Page scraped! Page Title:",soup.title.string)

Page scraped! Page Title: List of postal codes of Canada: M - Wikipedia


Create dataframe:

In [5]:
table = soup.find("table", attrs={"class": "wikitable sortable"})
table_data = table.find_all("tr")

In [6]:
df = []
for tr in table_data:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    df.append(row)

df = pd.DataFrame(df, columns=["Postal Code", "Borough", "Neighbourhood"])
df

,Postal Code,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
...,...,...,...
176,M5Z,Not assigned,Not assigned
177,M6Z,Not assigned,Not assigned
178,M7Z,Not assigned,Not assigned
179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Cleanup:

In [7]:
df.drop(index=0, inplace=True)
df

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
176,M5Z,Not assigned,Not assigned
177,M6Z,Not assigned,Not assigned
178,M7Z,Not assigned,Not assigned
179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Checked below is to see which rows 'Neighbourhood'=='Borough' as per instructor comment:  
>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  

In [8]:
assigned=df[df['Borough']==df['Neighbourhood']]
assigned.groupby('Borough').count()

,Postal Code,Neighbourhood
Borough,,
Not assigned,77,77


After check, all 'Borough' == 'Neighbourhood' have a value of 'Not assigned'. Will proceed to drop rows with 'Not assigned.

In [9]:
df = df[~df.Borough.str.contains("Not assigned")]
df = df[~df.Neighbourhood.str.contains("Not assigned")]
df

,Postal Code,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
df.reset_index(inplace=True, drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df.shape

(103, 3)

### Prepare data
#### PD frame should appear like the sample below:  
<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1593129600000&hmac=GsxpU6SLpg-AczZhIdIisV4u9msX1Ic0Er9MrF2ffSE" width=50%>  
  
#### To create the above dataframe:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.  

## 2. Get geo coordinates for postal codes
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.  
  
In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each   neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.  
  
The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:  

```python
import geocoder # import geocoder
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
```
Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

#### Your frame should appear like the sample below:
<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1593129600000&hmac=jrha7kwaFXVFSkFQV9XFmTgCRwvFrq12S1bY6JVak0M" width=50%>

In [12]:
import geocoder

Loop to add long and lat to df, used arcgis service to find values as it worked more reliably:

In [112]:
latitude=[]
longitude=[]

for i in df['Postal Code']:
    g = geocoder.arcgis('{},Toronto, Ontario'.format(i))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

df['Latitude'] = pd.DataFrame(latitude)
df['Longitude'] = pd.DataFrame(longitude)

df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


## 3. Generate maps
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure to add enough Markdown cells to explain what you decided to do and to report any observations you make, and generate maps to visualize your neighborhoods and how they cluster together.

In [113]:
import folium
from folium import plugins

In [114]:
# Capture Toronto lat long for use in map
g = geocoder.arcgis('Toronto, Ontario')
lat_lng_coords = g.latlng
latitude=lat_lng_coords[0]
longitude=lat_lng_coords[1]

In [122]:
# Create folium map
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 11)

# Cluster
nb = plugins.MarkerCluster().add_to(toronto_map)

# Add rows to cluster
for lat, lng, label, in zip(df.Latitude, df.Longitude, df.Borough):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(nb)

# Display map
toronto_map